# Desenvolvimento de um modelo de regressão logística para testar o retorno na API
Este notebook faz uma breve análise e apresenta uma proposta de modelo de classificação para o conjunto de dados obtidos do site **https://books.toscrape.com/**.
A proposta é tentar prever a **categoria de um livro**, baseado as nas varíaveis preço, disponibilidade e quantidade de estrelas.

## 1 - Carregamento dos dados

In [12]:
import pandas as pd
import os
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Carregando o arquivo CSV
path_csv = os.path.join(os.path.dirname(os.getcwd()), 'data', 'books_dataset.csv')
df_livros = pd.read_csv(path_csv, sep=";", encoding="utf-8")

# Inspecionar se os dados foram carregados corretamente
df_livros.head()

,id,categoria,url_categoria,titulo,link_livro,url_imagem,descricao_produto,qtde_estrelas,upc,tipo_produto,moeda,preco_excl_tax,preco_incl_tax,imposto,disponibilidade_produto,numero_de_reviews
0,0,Travel,https://books.toscrape.com/catalogue/category/...,It's Only the Himalayas,https://books.toscrape.com/catalogue/its-only-...,https://books.toscrape.com/media/cache/6d/41/6...,"“Wherever you go, whatever you do, just . . . ...",2,a22124811bfa8350,Books,£,45.17,45.17,0.0,19,0
1,1,Travel,https://books.toscrape.com/catalogue/category/...,Full Moon over Noah’s Ark: An Odyssey to Mount...,https://books.toscrape.com/catalogue/full-moon...,https://books.toscrape.com/media/cache/fe/8a/f...,Acclaimed travel writer Rick Antonson sets his...,4,ce60436f52c5ee68,Books,£,49.43,49.43,0.0,15,0
2,2,Travel,https://books.toscrape.com/catalogue/category/...,See America: A Celebration of Our National Par...,https://books.toscrape.com/catalogue/see-ameri...,https://books.toscrape.com/media/cache/c7/1a/c...,To coincide with the 2016 centennial anniversa...,3,f9705c362f070608,Books,£,48.87,48.87,0.0,14,0
3,3,Travel,https://books.toscrape.com/catalogue/category/...,Vagabonding: An Uncommon Guide to the Art of L...,https://books.toscrape.com/catalogue/vagabondi...,https://books.toscrape.com/media/cache/ca/30/c...,With a new foreword by Tim Ferriss •There’s no...,2,1809259a5a5f1d8d,Books,£,36.94,36.94,0.0,8,0
4,4,Travel,https://books.toscrape.com/catalogue/category/...,Under the Tuscan Sun,https://books.toscrape.com/catalogue/under-the...,https://books.toscrape.com/media/cache/45/21/4...,A CLASSIC FROM THE BESTSELLING AUTHOR OF UNDER...,3,a94350ee74deaa07,Books,£,37.33,37.33,0.0,7,0


## 2 - Análise exploratória

In [13]:
# Informações sobre o DataFrame
df_livros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       1000 non-null   int64  
 1   categoria                1000 non-null   object 
 2   url_categoria            1000 non-null   object 
 3   titulo                   1000 non-null   object 
 4   link_livro               1000 non-null   object 
 5   url_imagem               1000 non-null   object 
 6   descricao_produto        1000 non-null   object 
 7   qtde_estrelas            1000 non-null   int64  
 8   upc                      1000 non-null   object 
 9   tipo_produto             1000 non-null   object 
 10  moeda                    1000 non-null   object 
 11  preco_excl_tax           1000 non-null   float64
 12  preco_incl_tax           1000 non-null   float64
 13  imposto                  1000 non-null   float64
 14  disponibilidade_produto  

In [14]:
# Estatísticas descritivas do DataFrame
df_livros.describe()

,id,qtde_estrelas,preco_excl_tax,preco_incl_tax,imposto,disponibilidade_produto,numero_de_reviews
count,1000.000000,1000.000000,1000.00000,1000.00000,1000.0,1000.000000,1000.0
mean,499.500000,2.923000,35.07035,35.07035,0.0,8.585000,0.0
std,288.819436,1.434967,14.44669,14.44669,0.0,5.654622,0.0
min,0.000000,1.000000,10.00000,10.00000,0.0,1.000000,0.0
25%,249.750000,2.000000,22.10750,22.10750,0.0,3.000000,0.0
50%,499.500000,3.000000,35.98000,35.98000,0.0,7.000000,0.0
75%,749.250000,4.000000,47.45750,47.45750,0.0,14.000000,0.0
max,999.000000,5.000000,59.99000,59.99000,0.0,22.000000,0.0


**Diagnóstico da inspeção:**
As variáveis numéricas: `id`, `preco_excl_tax`, `imposto` e `numero_de_reviews` foram descartadas por não apresentar variabilidade e assim contribuirem muito pouco para a modelagem.
O restante das variáveis não tem aplicação para o escopo da proposta, com exceção da variável `categoria`, que será nossa dependente.

Variáveis que serão utilizadas:
- `qtde_estrelas`,
- `preco_incl_tax`,
- `disponibilidade_produto`,
- `categoria`

## 3 - Preparação dos dados

In [15]:
# Filtrando as colunas relevantes
features = ["qtde_estrelas", "preco_incl_tax", "disponibilidade_produto"]
target = "categoria"

df_ml = df_livros[features + [target]]

In [16]:
# Transformando a variável categórica 'categoria' em numérica
encoder = LabelEncoder()
df_ml[target] = encoder.fit_transform(df_ml[target])

# Verifica as primeiras linhas após a transformação
df_ml.head()

# Salvando o encoder para uso futuro
path_encoder = os.path.join(os.path.dirname(os.getcwd()), 'models', 'encoder.pkl')
joblib.dump(encoder, path_encoder)

C:\Users\rapha\AppData\Local\Temp\ipykernel_1080\2510254567.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ml[target] = encoder.fit_transform(df_ml[target])


['c:\\Users\\rapha\\OneDrive\\Documentos\\CURSOS\\FIAP - Machine Learning Engineering\\Fase 01\\Tech Challenge\\projeto_api_consulta_livros\\projeto\\models\\encoder.pkl']

In [17]:
# Separar X e y
X = df_ml[features]
y = df_ml[target]

# Dividir em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((700, 3), (300, 3), (700,), (300,))

## 4 - Treinamento do Modelo

In [18]:
# Treinando um modelo de regressão logística
model = LogisticRegression(max_iter=10000)
model.fit(X_train, y_train)


# Métricas de avaliação
score = model.score(X_test, y_test)
print(f"Acurácia do modelo é: {score:.2f}")

Acurácia do modelo é: 0.16


In [19]:
# Salvando o modelo treinado
path_model = os.path.join(os.path.dirname(os.getcwd()), 'models', 'modelo_bookscrape.pkl')
joblib.dump(model, path_model)
print("Modelo salvo em modelo_bookscrape.pkl")

Modelo salvo em modelo_bookscrape.pkl
